In [1]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.0


In [2]:
from tf_records_generator import get_dataset
from efficientNet_B0 import EffNet0
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [3]:
CONFIG = dict(
    replicas=1,
    input_shape=(256, 256, 3),
    batch_size=8,
    epochs=5,
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [4]:
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

In [5]:
# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)
CONFIG["replicas"] = strategy.num_replicas_in_sync

INFO:tensorflow:Initializing the TPU system: grpc://10.11.220.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.11.220.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.11.220.66:8470']
Number of accelerators:  8


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import PIL

AUTO = tf.data.experimental.AUTOTUNE


def show_dataset(thumb_size, cols, rows, ds):
    mosaic = PIL.Image.new(mode='RGB', size=(thumb_size * cols + (cols - 1),
                                             thumb_size * rows + (rows - 1)))
    for idx, data in enumerate(iter(ds)):
        img, target_or_imgid = data
        ix = idx % cols
        iy = idx // cols
        img = np.clip(img.numpy() * 255, 0, 255).astype(np.uint8)
        img = PIL.Image.fromarray(img)
        img = img.resize((thumb_size, thumb_size), resample=PIL.Image.BILINEAR)
        mosaic.paste(img, (ix * thumb_size + ix,
                           iy * thumb_size + iy))
    mosaic.show()


def map_function(img, imgname_or_label):
    return prepare_image(img), imgname_or_label


def get_dataset(files, config, augment=False, shuffle=False, repeat=False,
                labeled=True, return_image_names=True):
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    #ds = ds.cache()

    if repeat:
        ds = ds.repeat()

    if shuffle:
        ds = ds.shuffle(1024 * 8)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)

    ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)

    ds = ds.map(map_function,
                num_parallel_calls=AUTO)

    ds = ds.batch(160,drop_remainder=True)
    ds = ds.prefetch(AUTO)
    return ds


def prepare_image(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.reshape(img, [256,256,3])
    return img


def read_labeled_tfrecord(example):
    tfrec_format = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'image_name': tf.io.FixedLenFeature([], tf.string),
        'patient_id': tf.io.FixedLenFeature([], tf.int64),
        'sex': tf.io.FixedLenFeature([], tf.int64),
        'age_approx': tf.io.FixedLenFeature([], tf.int64),
        'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64),
        'diagnosis': tf.io.FixedLenFeature([], tf.int64),
        'target': tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    np.shape(example['image'])
    print(example['image'])
    return example['image'], example['target']


In [ ]:
import tensorflow.keras as keras
import tensorflow as tf


def EffNet0(config, trainable_base=False) -> keras.models.Sequential:
    conv_base = keras.applications.efficientnet.EfficientNetB0(weights='imagenet', include_top=False,
                                                               input_shape=(256,256,3))
    model = keras.models.Sequential()
    model.add(conv_base)
    model.add(keras.layers.GlobalMaxPooling2D(name="gap"))
    model.add(keras.layers.Dense(1, activation="sigmoid", name="fc_out"))
    conv_base.trainable = trainable_base
    return model


In [6]:
path = 'gs://dataset_files/256'
files_train = np.sort(np.array(tf.io.gfile.glob(path + '/train*.tfrec')))
ds = get_dataset(files_train, CONFIG)


Tensor("ParseSingleExample/ParseExample/ParseExampleV2:3", shape=(), dtype=string)


In [7]:
with strategy.scope():
  model = EffNet0(config=CONFIG, trainable_base=True)
  model.compile(optimizer=CONFIG["optimizer"], loss=CONFIG["loss"], metrics=['accuracy', keras.metrics.AUC()])



16711680/16705208 [==============================] - 0s 0us/step


In [8]:
history = model.fit(ds, batch_size=160, epochs=CONFIG["epochs"])

Epoch 1/5
204/204 [==============================] - 26s 129ms/step - loss: 0.1179 - accuracy: 0.9782 - auc: 0.6924
Epoch 2/5
204/204 [==============================] - 29s 143ms/step - loss: 0.0746 - accuracy: 0.9821 - auc: 0.8296
Epoch 3/5
204/204 [==============================] - 29s 141ms/step - loss: 0.0691 - accuracy: 0.9823 - auc: 0.8698
Epoch 4/5
204/204 [==============================] - 30s 145ms/step - loss: 0.0621 - accuracy: 0.9827 - auc: 0.9028
Epoch 5/5
204/204 [==============================] - 30s 146ms/step - loss: 0.0572 - accuracy: 0.9834 - auc: 0.9202
